In [1]:
import pandas as pd
import numpy as np
import keras
from keras.utils.data_utils import get_file
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from os.path import join
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


In [2]:
DATA_HOME = '../CASIA-WebFace-Cropped/'

In [3]:
dataset = pd.read_csv('webface.train.csv', nrows=None)

In [4]:
encoder = LabelEncoder()
encoder.fit(dataset['person'])
dataset['person_id'] = encoder.transform(dataset['person'])

In [5]:
dataset.head()

,person,count,path,person_id
0,3331486,13,3331486/012.jpg,1107
1,3331486,13,3331486/010.jpg,1107
2,3331486,13,3331486/006.jpg,1107
3,3331486,13,3331486/008.jpg,1107
4,3331486,13,3331486/001.jpg,1107


In [6]:
y = dataset['person_id'].as_matrix()

In [7]:
img_paths = [r.path for r in dataset.itertuples()]
print(len(img_paths))

65707


In [8]:
def path2ImgVec(path):
    x = img_to_array(load_img(join(DATA_HOME, path)))
    return x.reshape((1,) + x.shape)

In [9]:
pool = multiprocessing.Pool(8)
results = pool.map(path2ImgVec, img_paths)
pool.close()
pool.join()

In [10]:
X = np.vstack(results)

In [11]:
X.shape

(65707, 55, 47, 3)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten, Input, concatenate
from keras.utils import np_utils
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
from keras import metrics
from keras.callbacks import Callback
from keras.layers.normalization import BatchNormalization
from keras.engine import Model
from keras import optimizers

nb_class = len(np.unique(y))
print('nb_class', nb_class)
hidden_dim = 160
best_weights_filepath = '../models/best_weights.hdf5'

def build_model():
    image_input = Input(shape=X.shape[1:])
    
    conv1 = Conv2D(20, (4, 4), name='conv1')(image_input)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), name='pool1')(conv1)
#     pool1 = Dropout(rate=0.2)(pool1)
    
    conv2 = Conv2D(40, (3, 3), name='conv2')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), name='pool2')(conv2)
#     pool2 = Dropout(rate=0.2)(pool2)

    conv3 = Conv2D(60, (3, 3), name='conv3')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), name='pool3')(conv3)

    flat1 = Flatten(name='flat1')(pool3)
    
    conv4 = Conv2D(80, (2, 2), name='conv4')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    flat2 = Flatten(name='flat2')(conv4)
    
    merged = concatenate([flat1, flat2])
    
    out = Dense(hidden_dim, name='hidden1')(merged)
    out = BatchNormalization()(out)
    out = Activation('relu',name='deepid')(out)
    out = Dense(nb_class, activation='softmax', name='softmax_class')(out)
    
    model = Model(inputs=image_input, outputs=out)

    optimizer = optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=(1e-4))

    model.compile(
        optimizer=optimizer, #rmsprop
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    
    print(model.summary())
    return model

nb_class 2027


In [21]:
model = build_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 55, 47, 3)     0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 52, 44, 20)    980                                          
____________________________________________________________________________________________________
batch_normalization_6 (BatchNorm (None, 52, 44, 20)    80                                           
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 52, 44, 20)    0                                            
___________________________________________________________________________________________

In [22]:
saveBestModel = keras.callbacks.ModelCheckpoint(
    best_weights_filepath, 
    monitor='val_acc', 
    verbose=0, 
    save_best_only=True, 
    mode='auto'
)
earlyStopping=keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=1, 
    mode='auto'
)

model.fit(
    X_train, 
    y_train, 
    batch_size=512, 
    epochs=15,
    verbose=2, 
    validation_split=0.1, 
    shuffle=True,
    callbacks=[saveBestModel, earlyStopping],
)

Train on 53222 samples, validate on 5914 samples
Epoch 1/15
7s - loss: 7.1922 - acc: 0.0207 - val_loss: 7.3009 - val_acc: 0.0159
Epoch 2/15
6s - loss: 6.2929 - acc: 0.0651 - val_loss: 6.3670 - val_acc: 0.0758
Epoch 3/15
6s - loss: 5.4344 - acc: 0.1323 - val_loss: 5.6219 - val_acc: 0.1266
Epoch 4/15
6s - loss: 4.6244 - acc: 0.2222 - val_loss: 5.0970 - val_acc: 0.1747
Epoch 5/15
6s - loss: 3.8961 - acc: 0.3256 - val_loss: 4.7144 - val_acc: 0.2215
Epoch 6/15
6s - loss: 3.2542 - acc: 0.4296 - val_loss: 4.4904 - val_acc: 0.2624
Epoch 7/15
6s - loss: 2.7109 - acc: 0.5227 - val_loss: 4.2845 - val_acc: 0.2891
Epoch 8/15
6s - loss: 2.2514 - acc: 0.6008 - val_loss: 4.2216 - val_acc: 0.2966
Epoch 9/15
6s - loss: 1.8573 - acc: 0.6730 - val_loss: 4.2418 - val_acc: 0.3028
Epoch 10/15
6s - loss: 1.5283 - acc: 0.7355 - val_loss: 4.2024 - val_acc: 0.3176
Epoch 11/15
6s - loss: 1.2351 - acc: 0.7940 - val_loss: 4.1836 - val_acc: 0.3263
Epoch 12/15
6s - loss: 0.9903 - acc: 0.8419 - val_loss: 4.2370 - val_

In [23]:
model.load_weights(best_weights_filepath)

In [24]:
model.save('../models/webface-simple-cnn.3348.model.h5')

In [25]:
model.evaluate(X_test, y_test, batch_size=256, verbose=2)

[4.3723573990199069, 0.32186881759053876]

In [26]:
import gc
gc.collect()

0